In [ ]:
## このプログラムは「Dogs vs. Cats Redux: Kernels Edition」において
## 作成したノートブックで動作します

import pandas as pd
import os, zipfile
from sklearn.model_selection import train_test_split

def prepareData():
    """データを読み込んで前処理を行う
    
    Returns:
      train_df(Dataframe)   : 前処理後の訓練データ
      validate_df(Dataframe): 前処理後の検証データ
    """
    # 訓練データとテストデータの解凍
    # 解凍するzipファイル名
    data = ['train', 'test']

    # train.zip、test.zipをカレントディレクトリに展開
    for el in data:
        with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/' + el + ".zip", "r") as z:
            z.extractall(".")  # extract zip files to current dir

    # 訓練データのファイル名のdog.x.jpg、cat.x.jpgを使って1と0のラベルを生成
    # trainフォルダー内のファイル名を取得してfilenamesに格納
    filenames = os.listdir("./train")
    categories = []
    
    for filename in filenames:
        # ファイル名を分割して先頭要素(dog/cat)のみを取り出し、
        # dogは1、catは0をラベルにしてcategoryに格納
        category = filename.split('.')[0]
        if category == 'dog':
        # dogならラベル1として追加
            categories.append(1)
        else:
        # catならラベル0として追加
            categories.append(0)

    # dfの列filenameにファイル名filenamesを格納
    # 列categoriesにラベルの値categoryを格納
    df = pd.DataFrame({
        'filename': filenames,
        'category': categories
    })
    
    # 訓練データを訓練用と検証用に分ける
    # 訓練データの総数25000の10%を検証データにする
    train_df, validate_df = train_test_split(df, test_size=0.1)
    # 行インデックスを振り直す
    train_df = train_df.reset_index()
    validate_df = validate_df.reset_index()
       
    return train_df, validate_df

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def ImageDataGenerate(train_df, validate_df):
    """画像を加工処理する
    
    Returns:
      train_generator(DirectoryIterator):
          加工処理後の訓練データ
      validation_generator(DirectoryIterator):
          加工処理後の検証データ
    """
    # 画像をリサイズするサイズ
    img_width, img_height = 224, 224
    target_size = (img_width, img_height)
    # ミニバッチのサイズ
    batch_size = 16
    
    # ファイル名の列名とラベルの列名
    x_col, y_col = 'filename', 'category'
    # flow_from_dataframe()で画像を生成する際のclass_modeオプションの値
    # ジェネレーターが返すラベルの配列の形状として'binary'を格納
    class_mode = 'binary'

    # 訓練データを加工するジェネレーターを生成
    train_datagen = ImageDataGenerator(
        rotation_range=15,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        width_shift_range=0.1,
        height_shift_range=0.1
    )

    # flow_from_dataframe()の引数class_mode = "binary"の場合、
    # ラベルが格納されたtrain_dfのy_col = 'category'の列の値は
    # 文字列であることが必要なので、1と0の数値を文字列に変換しておく
    train_df['category'] = train_df['category'].astype(str)  #optional

    # ジェネレータを使って訓練データを生成
    train_generator = train_datagen.flow_from_dataframe(
        train_df,     # 訓練用のデータフレーム
        "./train/",   # 画像データのディレクトリ
        x_col=x_col,  # ファイル名が格納された列
        y_col=y_col,  # ラベルが格納された列 (文字列に変換済み)
        class_mode=class_mode,   # ラベルの配列の形状
        target_size=target_size, # 画像のサイズ
        batch_size=batch_size    # ミニバッチのサイズ
    )

    # 検証データを加工するジェネレーター
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # flow_from_dataframe()の引数class_mode = "binary"の場合、
    # ラベルが格納されたvalidate_dfのy_col = 'category'の列の値は
    # 文字列であることが必要なので、1と0の数値を文字列に変換しておく
    validate_df['category'] = validate_df['category'].astype(str)

    # ジェネレータを使って検証データを生成
    validation_generator = validation_datagen.flow_from_dataframe(
        validate_df,   # 検証用のデータフレーム
        "./train/",    # 画像データのディレクトリ
        x_col=x_col,   # ファイル名が格納された列
        y_col=y_col,   # ラベルが格納された列(文字列に変換済み)
        class_mode=class_mode,   # ラベルの配列の形状
        target_size=target_size, # 画像のサイズ
        batch_size=batch_size    # ミニバッチのサイズ
    )
    
    # 生成した訓練データと検証データを返す
    return train_generator, validation_generator

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalMaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import LearningRateScheduler
import math

def train_FClayer(train_generator, validation_generator):
    """ファインチューニングしたVGG16で学習する
    
    Returns:
      history(Historyオブジェクト)
    """
    # 画像のサイズを取得
    image_size = len(train_generator[0][0][0])
    # 入力データの形状をタプルにする
    input_shape = (image_size, image_size, 3)
    # ミニバッチのサイズを取得
    batch_size = len(train_generator[0][0])
    # 訓練データの数を取得(バッチの数×ミニバッチサイズ)
    total_train = len(train_generator)*batch_size
    # 検証データの数を取得(バッチの数×ミニバッチサイズ)
    total_validate = len(validation_generator)*batch_size

    # VGG16モデルを学習済みの重みと共に読み込む
    pre_trained_model = VGG16(
        include_top=False,            # 全結合層（FC）は読み込まない
        weights='imagenet',           # ImageNetで学習した重みを利用
        input_shape=input_shape   # 入力データの形状
    )

    for layer in pre_trained_model.layers[:15]:
        # 第1～第15層までの重みを凍結
        layer.trainable = False

    for layer in pre_trained_model.layers[15:]:
        # 第16層以降の重みを更新可能にする
        layer.trainable = True
    
    # Sequentualオブジェクトを生成
    model = Sequential()

    # VGG16モデルを追加
    model.add(pre_trained_model)

    # (batch_size, rows, cols, channels)の4階テンソルに
    # プーリング演算適用後、(batch_size, channels)の2階テンソルにフラット化
    model.add(
        GlobalMaxPooling2D())

    # 全結合層
    model.add(
        Dense(512,               # ユニット数512
              activation='relu') # 活性化関数はReLU
    )
    # 50%のドロップアウト
    model.add(Dropout(0.5))

    # 出力層
    model.add(
        Dense(1,                    # ユニット数1
              activation='sigmoid') # 活性化関数はSigmoid
    )
    
    # モデルのコンパイル
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1e-5),
                  metrics=['accuracy'])
    
    # コンパイル後のサマリを表示
    model.summary()

    # 学習率をスケジューリングする
    def step_decay(epoch):
        initial_lrate = 0.00001 # 学習率の初期値
        drop = 0.5              # 減衰率は50%
        epochs_drop = 10.0      # 10エポック毎に減衰する
        lrate = initial_lrate * math.pow(
            drop,
            math.floor((epoch)/epochs_drop)
        )
        return lrate

    # 学習率のコールバック
    lrate = LearningRateScheduler(step_decay)

    # ファインチューニングモデルで学習する
    epochs = 40   # エポック数
    history = model.fit(
        # 訓練データ
        train_generator,
        # エポック数
        epochs=epochs,
        # 訓練時のステップ数
        validation_data=validation_generator,
        # 検証データ
        validation_steps=total_validate//batch_size,
        # 検証時のステップ数
        steps_per_epoch=total_train//batch_size,
        # 学習の進捗状況を出力する    
        verbose=1,
        # 学習率のスケジューラーをコール
        callbacks=[lrate]
    )
    
    # historyを返す
    return history

In [ ]:
# 前処理したデータを取得
train_df, validate_df = prepareData()
# ジェネレーターで滑降する
train_generator, validation_generator = ImageDataGenerate(train_df, validate_df)
# VGG16の出力をFCネットワークで学習
history = train_FClayer(train_generator, validation_generator)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_acc_loss(history):
    # 精度の推移をプロット
    plt.plot(history.history['accuracy'],"-",label="accuracy")
    plt.plot(history.history['val_accuracy'],"-",label="val_acc")
    plt.title('accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()

    # 損失の推移をプロット
    plt.plot(history.history['loss'],"-",label="loss",)
    plt.plot(history.history['val_loss'],"-",label="val_loss")
    plt.title('loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='upper right')
    plt.show()
    
# 損失と精度をグラフに出力
plot_acc_loss(history)